In [ ]:
import sklearn
print(sklearn.__version__)


1.2.2


In [1]:
import cv2
import joblib
import numpy as np

# Load the trained models
mbk_model = joblib.load('mbk_model_final.pkl')
svm_model = joblib.load('svm_model_final.pkl')


cam = cv2.VideoCapture(0)
while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to read frame from camera")
        break
    frame = cv2.flip(frame, 1)
    cv2.rectangle(frame, (319, 9), (620 + 1, 309), (0, 255, 0), 1)
    roi = frame[10:300, 320:620]
    
    # cv2.imshow("Frame", frame)
    # des = extract_features(roi)
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    result = cv2.bitwise_and(roi, roi, mask=mask)

    # Convert image to grayscale
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)

    # Create SIFT object and find keypoints and descriptors
    sift = cv2.SIFT_create()
    kp, des = sift.detectAndCompute(gray, None)
    # pred = mbk.predict(des)
    # hist, _ = np.histogram(pred, bins=range(51))
    # # y_pred = clf.predict(hist)
    # y_pred = clf.predict(hist.reshape(1, -1))
    if des is not None:
        pred = mbk_model.predict(des)
        hist, _ = np.histogram(pred, bins=range(51))
        y_pred = svm_model.predict(hist.reshape(1, -1))
        # print(str(y_pred[0]))
        cv2.putText(frame, str(y_pred[0]), (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        # cv2.putText(frame, y_pred, (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)

        # print(alpha_dict[np.argmax(y_pred)])
        # cv2.putText(frame,y_pred, (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        # cv2.putText(frame, alpha_dict[np.argmax(y_pred)], (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    cv2.imshow("Frame", frame)
    k = cv2.waitKey(50) & 0xFF
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()

    